# **Running Jupyter Code with Ollama Server**

1. **Download and Install Ollama**: Ensure that Ollama is downloaded and installed on your system.

2. **Check Ollama Version**:
   ```bash
   ollama --version
   ```

3. **Run Llama 3.2 Model**:
   To load the Llama 3.2 model:
   ```bash
   ollama run llama3.2
   ```

4. **Start Ollama Server**:
   To start the Ollama server, run:
   ```bash
   ollama serve
   ```

**Note**: If you face any issues while running the server, checking the background tasks using the port can help resolve conflicts. Always ensure the port is free before starting the server.

1. **Check if Ollama Server is Running**:
   Check if the server is running and using port 11434:
   ```bash
   netstat -ano | findstr :11434
   ```

2. **Identify and Kill Background Task (if needed)**:
   If you see that another task is using the port, you can identify and kill it:
   ```bash
   tasklist /FI "PID eq <PID>"
   taskkill /PID <PID> /F
   ```

3. **Close Ollama from Windows Task Bar (Optional)**:
   If the server is still not responding or you see issues, you can use the **up arrow key** (^) on the Windows task bar (bottom-left) to close any running Ollama instances.

4. **Start Jupyter Notebook**:
   Once the server is running, start your Jupyter notebook with the necessary code to interact with Ollama.

In [41]:
import ollama
from PyPDF2 import PdfReader
from IPython.display import display, Markdown ,update_display

In [42]:
system_prompt = '''You are an agent which analyses the resume and understands the context of resume and 
    extract the following fields and format the output:
    - Name
    - Email
    - Phone 1
    - Phone 2
    - Address
    - City
    - LinkedIn
    - Professional Experience (in years)
    - Highest Education
    - Is Fresher (yes/no)
    - Is Student (yes/no)
    - Skills (comma-separated)
    - Applied For Profile
    - Education (Institute Name, Year of Passing, Score)
    - Projects (Project Name, Description)
    - Professional Experience (Organisation Name, Duration, Profile)

    Ensure to format the output as:
    Name: [Value]
    Email: [Value]
    Phone 1: [Value]
    ... and provide this summary of resume in markdown format'''

In [43]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()  # Extract text from each page
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text
    
user_prompt = extract_text_from_pdf('./data/python-developer-resume-example.pdf') + "Provide a summary of this resume"

## **Complete Response by Ollama**

In [38]:
import ollama

def call_ollama(system_prompt, user_prompt):
    content = ollama.chat(model='llama3.2', messages=[
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': user_prompt
        }]
        )
    
    print(content)
    
# # Example usage
system_prompt = "You are a helpful assistant"
user_prompt = "What is captial of India"

call_ollama(system_prompt, user_prompt)

model='llama3.2' created_at='2025-01-16T15:52:03.1486778Z' done=True done_reason='stop' total_duration=662937700 load_duration=28479100 prompt_eval_count=36 prompt_eval_duration=81336000 eval_count=9 eval_duration=549193000 message=Message(role='assistant', content='The capital of India is New Delhi.', images=None, tool_calls=None)


## **Chunk Response by Ollama**

**NOTE :** If we use stream=True parameter model gives response in chunks

In [51]:
import ollama

def call_ollama(system_prompt, user_prompt):
    content = ollama.chat(model='llama3.2', messages=[
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': user_prompt
        }],
        stream=True
        )
    
    for chunk in content:
        print(chunk)
    
# # Example usage
system_prompt = "You are a helpful assistant"
user_prompt = "What is captial of India"

call_ollama(system_prompt, user_prompt)

model='llama3.2' created_at='2025-01-16T16:08:15.4735779Z' done=False done_reason=None total_duration=None load_duration=None prompt_eval_count=None prompt_eval_duration=None eval_count=None eval_duration=None message=Message(role='assistant', content='The', images=None, tool_calls=None)
model='llama3.2' created_at='2025-01-16T16:08:15.5452448Z' done=False done_reason=None total_duration=None load_duration=None prompt_eval_count=None prompt_eval_duration=None eval_count=None eval_duration=None message=Message(role='assistant', content=' capital', images=None, tool_calls=None)
model='llama3.2' created_at='2025-01-16T16:08:15.6105445Z' done=False done_reason=None total_duration=None load_duration=None prompt_eval_count=None prompt_eval_duration=None eval_count=None eval_duration=None message=Message(role='assistant', content=' of', images=None, tool_calls=None)
model='llama3.2' created_at='2025-01-16T16:08:15.6782778Z' done=False done_reason=None total_duration=None load_duration=None pr

## **Streaming with Markdown**

In [49]:
import ollama

def call_ollama(system_prompt, user_prompt):
    stream = ollama.chat(model='llama3.2', messages=[
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': user_prompt
        }],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content']
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response),display_id=display_handle.display_id)

# Example usage
system_prompt = "You are a helpful assistant"
# user_prompt = "What is the capital of India?"

call_ollama(system_prompt, user_prompt)

Here's a summary of the resume:

**Summary:**
This is a professional resume for a Python developer with experience working on various projects, including web applications and data pipelines. The candidate has a strong background in computer science, with degrees from University of Chicago and University of Pittsburgh.

**Key Skills:**

* Programming languages: HTML/CSS, SQL (PostgreSQL, Oracle), JavaScript (Angular), Python (Django)
* Frameworks: Angular, Django
* Databases: PostgreSQL, Oracle
* Cloud platforms: AWS (Redshift, S3)
* Version control: Git

**Work Experience:**

* Current role as a Python Developer at DoorDash, where the candidate has worked on building new features and improving user experience.
* Previous internship at Knewton, where they contributed to implementing RESTful APIs in Django and built unit testing infrastructure for a client web application.

**Projects:**

* Cryptocurrency Price Tracker, which incorporated API calls to several applications and utilized D3.js for dynamic visualization.

**Education:**

* M.S. Computer Science, University of Chicago (2014-2016)
* B.S. Computer Science, University of Pittsburgh (2010-2014)

Overall, this resume showcases the candidate's technical skills, experience working on various projects, and their ability to improve user experience and contribute to data-driven decision making.